In [63]:

import os
import time
import urllib

from dotenv import load_dotenv
import pandas
import requests 
from tenacity import retry, stop_after_attempt, wait_exponential

# response = requests.get(call_text, headers=auth_header)
# df = pandas.DataFrame(response.json()['value'])

# next_link =  response.json().get('@odata.nextLink')
# while next_link:
# 	time.sleep(30)
# 	response = requests.get(next_link, headers=auth_header)
# 	print(next_link)
# 	df = pandas.concat([df, pandas.DataFrame(response.json()['value'])])
# 	print(df.shape)
load_dotenv()

# @retry(stop=stop_after_attempt(3), wait=wait_exponential(min=1)) # it waits 2^x * 1 second between each retry; min=1 waits 1s to start retries
def call_property_endpoint(url, headers, params):
    response = requests.get(url, headers=headers, params=params) # type(response.content) = <class 'bytes'>

    response.raise_for_status()
    return response.json() # type(response.json()) = <class 'dict'>

url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"
headers = {
    'x-rapidapi-key': os.getenv("API_KEY"),
    'x-rapidapi-host': "zillow-com1.p.rapidapi.com"
}
params = {
	"location": "Chicago, IL",
	"status_type": "ForSale",
	"home_type": "Houses, Townhomes",
	"minPrice": "400000",
	"maxPrice": "450000",
	"bedsMin": "3",
}

current_page = 1
response = call_property_endpoint(url, headers, params)
if response:
	df = pandas.DataFrame(pandas.json_normalize(response['props'])) 
df

,dateSold,carouselPhotos,has3DModel,propertyType,lotAreaValue,lotAreaUnit,address,detailUrl,variableData,priceChange,...,contingentListingType,imgSrc,bathrooms,country,currency,livingArea,hasImage,listingSubType.is_FSBA,listingSubType.is_openHouse,unit
0,None,[{'url': 'https://photos.zillowstatic.com/fp/8...,False,SINGLE_FAMILY,4500.000,sqft,"3323 W Belden Ave, Chicago, IL 60647",/homedetails/3323-W-Belden-Ave-Chicago-IL-6064...,None,NaN,...,None,https://photos.zillowstatic.com/fp/8d12e535cc7...,3,USA,USD,1300,True,True,NaN,NaN
1,None,[{'url': 'https://photos.zillowstatic.com/fp/d...,False,SINGLE_FAMILY,3125.000,sqft,"3817 N Kilpatrick Ave, Chicago, IL 60641",/homedetails/3817-N-Kilpatrick-Ave-Chicago-IL-...,None,NaN,...,None,https://photos.zillowstatic.com/fp/dbafc40b9bc...,2,USA,USD,0,True,True,True,NaN
2,None,[{'url': 'https://photos.zillowstatic.com/fp/2...,False,SINGLE_FAMILY,3123.252,sqft,"4322 N Kimball Ave, Chicago, IL 60618",/homedetails/4322-N-Kimball-Ave-Chicago-IL-606...,None,-40000.0,...,None,https://photos.zillowstatic.com/fp/2929bb19f13...,3,USA,USD,1200,True,True,NaN,NaN
3,None,[{'url': 'https://photos.zillowstatic.com/fp/0...,False,SINGLE_FAMILY,3625.000,sqft,"1119 N Avers Ave, Chicago, IL 60651",/homedetails/1119-N-Avers-Ave-Chicago-IL-60651...,None,NaN,...,None,https://photos.zillowstatic.com/fp/05da5c2298c...,3,USA,USD,1483,True,True,NaN,NaN
4,None,[{'url': 'https://photos.zillowstatic.com/fp/0...,False,SINGLE_FAMILY,3123.252,sqft,"3716 N Mozart St, Chicago, IL 60618",/homedetails/3716-N-Mozart-St-Chicago-IL-60618...,None,NaN,...,None,https://photos.zillowstatic.com/fp/055c842d82e...,3,USA,USD,1800,True,True,NaN,NaN
5,None,[{'url': 'https://photos.zillowstatic.com/fp/d...,False,SINGLE_FAMILY,3149.388,sqft,"1313 N Artesian Ave, Chicago, IL 60622",/homedetails/1313-N-Artesian-Ave-Chicago-IL-60...,None,NaN,...,None,https://photos.zillowstatic.com/fp/d5f77b60163...,1,USA,USD,1354,True,True,NaN,NaN
6,None,[{'url': 'https://photos.zillowstatic.com/fp/9...,False,SINGLE_FAMILY,3125.000,sqft,"2736 W Wilcox St, Chicago, IL 60612",/homedetails/2736-W-Wilcox-St-Chicago-IL-60612...,None,-13000.0,...,None,https://photos.zillowstatic.com/fp/9637dbfa41a...,4,USA,USD,2755,True,True,NaN,NaN
7,None,[{'url': 'https://photos.zillowstatic.com/fp/b...,True,SINGLE_FAMILY,3125.000,sqft,"4116 N Central Ave, Chicago, IL 60634",/homedetails/4116-N-Central-Ave-Chicago-IL-606...,None,NaN,...,None,https://photos.zillowstatic.com/fp/baf1d9dd668...,2,USA,USD,0,True,True,NaN,NaN
8,None,[{'url': 'https://photos.zillowstatic.com/fp/6...,False,SINGLE_FAMILY,6185.520,sqft,"9846 S Hamilton Ave, Chicago, IL 60643",/homedetails/9846-S-Hamilton-Ave-Chicago-IL-60...,None,-40000.0,...,None,https://photos.zillowstatic.com/fp/660af64c1e4...,2,USA,USD,2307,True,True,NaN,NaN
9,None,[{'url': 'https://photos.zillowstatic.com/fp/0...,False,SINGLE_FAMILY,4477.968,sqft,"5939 W Touhy Ave, Chicago, IL 60646",/homedetails/5939-W-Touhy-Ave-Chicago-IL-60646...,None,NaN,...,None,https://photos.zillowstatic.com/fp/00e8b1800d5...,2,USA,USD,1214,True,True,NaN,NaN


In [64]:
while current_page <= response['totalPages']:
	current_page += 1
	print(f"Making get request for page {current_page}")
	params['page'] = current_page
	try:
		response = call_property_endpoint(url, headers=headers, params=params)
		print(f"Current response record count: {response['resultsPerPage']}")
	except Exception as err:
		print(f"Error when making the get request: {err.__repr__()}")
	if response:
		addl_df = pandas.DataFrame(pandas.json_normalize(response['props'])) 
		df = pandas.concat([df, addl_df], ignore_index=True) # sub-frame indices are not relevant here
df

Making get request for page 2
Current response record count: 41
Making get request for page 3
Current response record count: 41


,dateSold,carouselPhotos,has3DModel,propertyType,lotAreaValue,lotAreaUnit,address,detailUrl,variableData,priceChange,...,contingentListingType,imgSrc,bathrooms,country,currency,livingArea,hasImage,listingSubType.is_FSBA,listingSubType.is_openHouse,unit
0,None,[{'url': 'https://photos.zillowstatic.com/fp/8...,False,SINGLE_FAMILY,4500.000,sqft,"3323 W Belden Ave, Chicago, IL 60647",/homedetails/3323-W-Belden-Ave-Chicago-IL-6064...,None,NaN,...,None,https://photos.zillowstatic.com/fp/8d12e535cc7...,3,USA,USD,1300,True,True,NaN,NaN
1,None,[{'url': 'https://photos.zillowstatic.com/fp/d...,False,SINGLE_FAMILY,3125.000,sqft,"3817 N Kilpatrick Ave, Chicago, IL 60641",/homedetails/3817-N-Kilpatrick-Ave-Chicago-IL-...,None,NaN,...,None,https://photos.zillowstatic.com/fp/dbafc40b9bc...,2,USA,USD,0,True,True,True,NaN
2,None,[{'url': 'https://photos.zillowstatic.com/fp/2...,False,SINGLE_FAMILY,3123.252,sqft,"4322 N Kimball Ave, Chicago, IL 60618",/homedetails/4322-N-Kimball-Ave-Chicago-IL-606...,None,-40000.0,...,None,https://photos.zillowstatic.com/fp/2929bb19f13...,3,USA,USD,1200,True,True,NaN,NaN
3,None,[{'url': 'https://photos.zillowstatic.com/fp/0...,False,SINGLE_FAMILY,3625.000,sqft,"1119 N Avers Ave, Chicago, IL 60651",/homedetails/1119-N-Avers-Ave-Chicago-IL-60651...,None,NaN,...,None,https://photos.zillowstatic.com/fp/05da5c2298c...,3,USA,USD,1483,True,True,NaN,NaN
4,None,[{'url': 'https://photos.zillowstatic.com/fp/0...,False,SINGLE_FAMILY,3123.252,sqft,"3716 N Mozart St, Chicago, IL 60618",/homedetails/3716-N-Mozart-St-Chicago-IL-60618...,None,NaN,...,None,https://photos.zillowstatic.com/fp/055c842d82e...,3,USA,USD,1800,True,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,None,[{'url': 'https://photos.zillowstatic.com/fp/5...,False,SINGLE_FAMILY,3810.000,sqft,"3531 N Nottingham Ave, Chicago, IL 60634",/homedetails/3531-N-Nottingham-Ave-Chicago-IL-...,None,NaN,...,None,https://photos.zillowstatic.com/fp/540140d45d7...,3,USA,USD,2080,True,True,NaN,NaN
108,None,[{'url': 'https://photos.zillowstatic.com/fp/3...,False,SINGLE_FAMILY,3920.400,sqft,"5044 W Wilson Ave, Chicago, IL 60630",/homedetails/5044-W-Wilson-Ave-Chicago-IL-6063...,None,NaN,...,None,https://photos.zillowstatic.com/fp/349c9b83829...,2,USA,USD,1600,True,True,NaN,NaN
109,None,[{'url': 'https://photos.zillowstatic.com/fp/9...,False,TOWNHOUSE,1020.000,sqft,"3383 S Archer Ave, Chicago, IL 60608",/homedetails/3383-S-Archer-Ave-Chicago-IL-6060...,None,NaN,...,None,https://photos.zillowstatic.com/fp/98bdc3dd615...,3,USA,USD,2030,True,True,True,NaN
110,None,[{'url': 'https://photos.zillowstatic.com/fp/4...,False,TOWNHOUSE,1200.000,sqft,"1823 S Melody Ct, Chicago, IL 60616",/homedetails/1823-S-Melody-Ct-Chicago-IL-60616...,None,NaN,...,None,https://photos.zillowstatic.com/fp/49918516f52...,4,USA,USD,2400,True,True,NaN,NaN


In [67]:
# filter based on area; eg: east of Cicero Ave
df = df[df['longitude'].le(87.7450)]  
# remove from set if contingent or on the market for too long
df = df[df['contingentListingType'].notnull() | df['daysOnZillow'].lt(180)]
# add column to mark records of particular interest
df['estimateDifference'] = df['zestimate'] - df['price']
df['specialInterest'] = (df['priceChange'] < 10000) | df['estimateDifference'].ge(10000)

#df.to_csv("data.txt", sep='\t')
df[['zestimate', 'price', 'priceChange', 'estimateDifference', 'specialInterest']].head(30)

,zestimate,price,priceChange,estimateDifference,specialInterest
2,NaN,445000,-40000.0,NaN,True
4,NaN,449000,NaN,NaN,False
51,439000.0,439900,-25000.0,-900.0,True
73,NaN,445000,-40000.0,NaN,True
75,NaN,449000,NaN,NaN,False


In [69]:
len(df)

5